# Project: Performance of phenotype algorithms for the identification of opioid-exposed infants, Andrew D. Wiese et al. Hospital Pediatrics 2024
# Title: Exclude infants with evidence of respiratory procedures
# Summary: 
## Exclude infants with evidence of respiratory procedures based on ICD procedure codes

# Notes:
- Look for evidence of respiratory procedures during birth hospitalization stay


##### Algorithm steps:
```
1. Define list of respiratory procedure codes (ICD-9 and ICD-10)
2. Find records in procedure_occurrence table matching respiratory codes
3. Filter records to baby persons only
4. Join with first baby visit data to get visit start/end dates
5. Filter procedure records to those occurring during first visit 
6. Register final view of baby respiratory procedures
```

##### Data Dictionaries:

**respiratory_code_list** - List of ICD-9 and ICD-10 procedure codes indicating respiratory procedures

**proc_df** - DataFrame of procedure records matching respiratory codes for all persons

**mom_baby_step3_respiratory** - Final DataFrame of baby respiratory procedures occurring during first visit 

##### Usage Notes:
```
- Requires CDM procedure_occurrence table as input
- Joins with first baby visit data from mom_baby_step1_baby1stvisit view
- Final view registered as global_temp.mom_baby_step3_respiratory
- Validation done by inspecting final DataFrame
```

In [0]:
%run "./project_modules"

##### Create view of respiratory procedure codes 
##### 1. respiratory procedure code:
- Insertion of an endotraceal tube: ICD-9 procedure code 96.04; ICD-10-PCS codes 0BH17EZ, 0BH18EZ 
- Invasive Mechanical ventilation: ICD-9 procedure codes 96.70, 96.71, 96.72; ICD-10-PCS codes 5A1935Z, 5A1945Z, 5A1955Z 
- Non-invasive mechnical ventilation: ICD-9 procedure code 93.90; ICD-10-PCS codes 5A09357, 5A09457, 5A09557
##### 2. Find records in procedure_occurrence table
##### 3. baby only


In [0]:
proc_df=mon_baby_records_code_list('procedure','fact_id_2',respiratory_code_list)
proc_df.createOrReplaceTempView("proc_df")

sql="""
    select  distinct proc_df.*,c.first_visit_start_date,c.first_visit_end_date from 
    proc_df
    inner join 
    global_temp.mom_baby_step1_baby1stvisit c
    on proc_df.baby_person_id = c.baby_person_id
    where code_date >= first_visit_start_date and code_date<= first_visit_end_date;
    """
mom_baby_step3_respiratory = spark.sql(sql)
mom_baby_step3_respiratory.name='mom_baby_step3_respiratory'
register_parquet_global_view(mom_baby_step3_respiratory)

##### Validation

In [0]:
df_inspection("global_temp.mom_baby_step3_respiratory","all")